Notebook to take into account the relative abundances of species when calculating the average number of gene copies across species of a same genus / family.
- data too large to be included in this Git repository 
- data available on demand: albane.ruaud@tuebingen.mpg.de

# Preambule

In [1]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(qs)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


qs v0.23.4.



# Taxa

In [2]:
# taxa used in the model
feats <- qread('../data/taxa_not_flt.qs') %>%
    select(-Sample)
feats <- data.frame(Feature = colnames(feats), Abundance_avg = colMeans(feats))

In [3]:
feats <- feats %>% subset(Abundance_avg > 0 ) 
feats %>% dim

[1] 4733    2

In [4]:
feats %>% head(2)

,Feature,Abundance_avg
,<chr>,<dbl>
s_1_14_0_10_34_76_sp002763075,s_1_14_0_10_34_76_sp002763075,6.173400e-07
s_1_19_sp001922585,s_1_19_sp001922585,4.584657e-07


# llcds results

In [5]:
hits <- fread('/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/llcds_fungene/annotate//dmnd_hits_all.tsv.gz')

In [6]:
# filtering
hits_f <- hits %>% subset(pident >= 50 & length / qlen > 0.8)

In [7]:
hits %>% dim
hits_f %>% dim

[1] 2971470      15

[1] 379393     15

# Add taxonomic level

In [8]:
aa_files_f <- read.table('/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/samples.tsv'
                        , sep = '\t', header = TRUE)  %>%
    select(Sample, Phylum, Class, Order, Family, Genus, Species) %>%
    rename('Genome' = Sample)

In [9]:
# all taxa lacking hits
hits_f <- aa_files_f %>%
    left_join(hits_f, by = 'Genome')
hits_f$V5[is.na(hits_f$V5)] <- 'None'

In [10]:
hits_f %>% colnames

[1] "Genome"   "Phylum"   "Class"    "Order"    "Family"   "Genus"   
 [7] "Species"  "qseqid"   "sseqid"   "pident"   "length"   "mismatch"
[13] "gapopen"  "qstart"   "qend"     "sstart"   "send"     "evalue"  
[19] "bitscore" "qlen"     "V5"

In [11]:
hits_tax <- hits_f %>% select(Genome, Phylum, Class, Order, Family, Genus, Species, V5, pident) %>% 
                subset(Species %in% feats$Feature)

In [12]:
dim(hits_tax)

[1] 90251     9

# Weight by genome size

In [13]:
samps_tbl_file <- read.table(header = TRUE, sep = '\t', dec = '.', fill = TRUE
 , '/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/samples.tsv')

In [14]:
struo_meta <- fread('/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv')

In [15]:
struo_meta$Genome <- str_replace(struo_meta$gtdb_genome_representative, pattern = '\\.', replacement = '_')

In [16]:
hits_tax$Genome_short <- str_extract(hits_tax$Genome, pattern = '^[:upper:]+\\_[:upper:]+\\_[:digit:]+\\_[:digit:]+')

In [17]:
sum(hits_tax$Genome_short %in% unique(struo_meta$Genome))
length(hits_tax$Genome)

[1] 90251

[1] 90251

In [18]:
hits_tax <- left_join(hits_tax, select(struo_meta, Genome, genome_size), by = c('Genome_short' = 'Genome'))

In [19]:
dim(hits_tax)

[1] 90251    11

# Per taxonomic level

## species

In [20]:
pp <- hits_tax %>% group_by(Phylum, Class, Order, Family, Genus, Species, V5) %>% 
        summarise(n_copies = n(), pident = mean(pident, na.rm = TRUE), genome_size = unique(genome_size))

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species' (override with `.groups` argument)



In [21]:
pp$r_copies <- pp$n_copies/pp$genome_size
pp <- select(pp, -n_copies, -genome_size)

## add all missing categories for each species

In [22]:
sp <- pp$Species %>% unique
hctg <- pp$V5 %>% unique

In [23]:
# create all combinations of species and acetobase gene needed
to_add <- expand.grid(sp, hctg) %>% as.data.frame
colnames(to_add) <- c('Species', 'V5')
    
# keep only the ones missing: give them a 0 count number and NA pident
to_add <- setdiff(to_add, pp %>% ungroup %>% select(Species, V5)) %>% 
                # add the taxonomic levels
                left_join(pp %>% ungroup %>% select(Phylum, Class, Order,Family, Genus, Species) %>% unique, by = 'Species') 
to_add$r_copies <- 0
to_add$pident <- NaN
    
# add them to the dataset
pp <- rbind(pp, to_add)

In [24]:
pp %>% tail(2)

Phylum,Class,Order,Family,Genus,Species,V5,pident,r_copies
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,g_Akkermansia,s_Akkermansia_sp001580195,mcrA,NaN,0
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,g_Akkermansia,s_Akkermansia_sp002358905,mcrA,NaN,0


In [25]:
pp <- pp %>% subset(V5 != 'None')

## genera

In [26]:
gen <- pp %>% left_join(feats, by = c('Species' = 'Feature')) %>% 
        group_by(Phylum, Class, Order, Family, Genus, V5) %>% 
        summarise(r_copies = weighted.mean(x = r_copies, w = Abundance_avg, na.rm = TRUE)
                  , pident =  weighted.mean(x = pident, w = Abundance_avg, na.rm = TRUE))
gen$Feature <- gen$Genus 

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family', 'Genus' (override with `.groups` argument)



## family

In [27]:
fam <- pp %>% left_join(feats, by = c('Species' = 'Feature')) %>% 
        group_by(Phylum, Class, Order, Family, V5) %>% 
        summarise(r_copies = weighted.mean(x = r_copies, w = Abundance_avg, na.rm = TRUE)
                  , pident =  weighted.mean(x = pident, w = Abundance_avg, na.rm = TRUE))
fam$Feature <- fam$Family

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family' (override with `.groups` argument)



## merge

In [28]:
colnames(pp)
colnames(gen)
colnames(fam)

[1] "Phylum"   "Class"    "Order"    "Family"   "Genus"    "Species"  "V5"      
[8] "pident"   "r_copies"

[1] "Phylum"   "Class"    "Order"    "Family"   "Genus"    "V5"       "r_copies"
[8] "pident"   "Feature"

[1] "Phylum"   "Class"    "Order"    "Family"   "V5"       "r_copies" "pident"  
[8] "Feature"

In [29]:
gen$Species <- NA
fam$Genus <- NA
fam$Species <- NA
pp$Feature <- pp$Species

In [30]:
nrow(pp)
nrow(gen)
nrow(fam)

[1] 17646

[1] 5646

[1] 1098

In [31]:
pp <- union(pp, gen) %>% union(fam)

In [32]:
nrow(pp)

[1] 24390

In [33]:
pp %>% head(3)
pp %>% tail(3)

Phylum,Class,Order,Family,Genus,Species,V5,pident,r_copies,Feature
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
p_Acidobacteriota,c_Acidobacteriae,o_Acidobacteriales,f_Koribacteraceae,g_QIAA01,s_QIAA01_sp003224905,acs,54.20,1.772694e-06,s_QIAA01_sp003224905
p_Acidobacteriota,c_Acidobacteriae,o_Acidobacteriales,f_Koribacteraceae,g_QIAA01,s_QIAA01_sp003224905,dsrB,68.90,7.090777e-07,s_QIAA01_sp003224905
p_Acidobacteriota,c_Acidobacteriae,o_Acidobacteriales,f_Koribacteraceae,g_QIAA01,s_QIAA01_sp003224905,hydA,63.09,1.772694e-06,s_QIAA01_sp003224905


Phylum,Class,Order,Family,Genus,Species,V5,pident,r_copies,Feature
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,NA,NA,dsrB,NaN,0.000000e+00,f_Akkermansiaceae
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,NA,NA,hydA,52.79006,3.955956e-06,f_Akkermansiaceae
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,NA,NA,mcrA,NaN,0.000000e+00,f_Akkermansiaceae


In [34]:
(!(unique(pp$Feature) %in% feats$Feature)) %>% sum

[1] 0

# Formatting

In [35]:
colnames(pp)[colnames(pp) == 'V5'] <- 'dmnd_ctg'

In [36]:
dmnd_pergene <- pp %>% ungroup %>% select(-Phylum, -Class, -Order, -Family, -Genus, -Species)

In [37]:
feat_imp <- qread('../tmp/Variable_importance.qs')
dmnd_pergene <- dmnd_pergene %>% left_join(feat_imp, by = 'Feature')

In [38]:
unique(dmnd_pergene$dmnd_ctg ) %>% sort

[1] "acs"  "cooS" "dsrA" "dsrB" "hydA" "mcrA"

In [39]:
dmnd_pergene <- dmnd_pergene %>% subset(dmnd_ctg %in% c('dsrA', 'dsrB'))
dmnd_pergene$pred_general <- 'SRB'

In [40]:
head(dmnd_pergene)

dmnd_ctg,pident,r_copies,Feature,Gini,importance,pred_general
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
dsrB,68.90000,7.090777e-07,s_QIAA01_sp003224905,NA,NA,SRB
dsrB,50.00000,2.977571e-07,s_OLB17_sp002360555,NA,NA,SRB
dsrB,53.33333,1.188200e-06,s_F0332_sp001652275,NA,NA,SRB
dsrB,57.27500,1.932138e-06,s_Gleimia_europaea_A,NA,NA,SRB
dsrB,57.27500,2.054550e-06,s_Gleimia_urinae,NA,NA,SRB
dsrB,52.90000,4.385363e-07,s_Varibaculum_cambriense_A,NA,NA,SRB


In [41]:
qsave(dmnd_pergene, 'Fungene_dmnd_pergene_r.qs')